# New York City Health Inspection Record Quick Analysis





I downloaded this dataset on July 7th from https://nycopendata.socrata.com/Health/DOHMH-New-York-City-Restaurant-Inspection-Results/xx67-kt59. This dataset has information of violations, grades, and inspection details on New York City restaurants throughout the five boroughs. 

In [1]:
import pandas as pd
import sqlite3
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

csv_filepath = 'your_dataset_location_here'
food = pd.read_csv(csv_filepath)

To begin the analysis, we must clean the dataset and remove any missing values.

In [23]:
# Drop rows with negative values
food = food[food['SCORE'] > 0]

# Drop rows with missing values in borough, grade, and score
food = food.dropna(subset = ['GRADE', 'SCORE'] )
food = food[food['BORO'] != 'Missing']

# Change the grades 'P' and 'Z' into 'Pending'
food.replace({'GRADE': {'P' : 'Pending', 'Z' : 'Pending'}}, inplace=True)

# Show result
food.head(5)

,﻿CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINE DESCRIPTION,INSPECTION DATE,ACTION,VIOLATION CODE,VIOLATION DESCRIPTION,CRITICAL FLAG,SCORE,GRADE,GRADE DATE,RECORD DATE,INSPECTION TYPE
0,41628429,THE COFFEE BEAN & TEA LEAF,MANHATTAN,392,COLUMBUS AVENUE,10024,2127991444,CafÃ©/Coffee/Tea,09/11/2013,Violations were cited in the following area(s).,10B,Plumbing not properly installed or maintained;...,Not Critical,8,A,09/11/2013,07/07/2016,Cycle Inspection / Re-inspection
1,41652187,GOLDEN CORNER NOODLES,QUEENS,4215,COLLEGE POINT BOULEVARD,11355,7188860123,Chinese,02/05/2014,Violations were cited in the following area(s).,06E,"Sanitized equipment or utensil, including in-u...",Critical,21,B,02/05/2014,07/07/2016,Cycle Inspection / Re-inspection
2,50000945,Khan's Gyro & Fried Chicken,BROOKLYN,1947,BATH AVE,11214,3473127777,American,10/22/2013,Violations were cited in the following area(s).,04M,Live roaches present in facility's food and/or...,Critical,25,B,10/22/2013,07/07/2016,Cycle Inspection / Re-inspection
5,41633990,DUNKIN' DONUTS,BROOKLYN,306,7 AVENUE,11215,3475295253,Donuts,06/12/2013,Violations were cited in the following area(s).,10F,Non-food contact surface improperly constructe...,Not Critical,2,A,06/12/2013,07/07/2016,Cycle Inspection / Initial Inspection
6,50034982,YOUR WAY CAFE,BROOKLYN,17,WILSON AVE,11237,3474068510,CafÃ©/Coffee/Tea,06/05/2015,Violations were cited in the following area(s).,04M,Live roaches present in facility's food and/or...,Critical,16,B,06/05/2015,07/07/2016,Pre-permit (Operational) / Re-inspection


Now let's take a glance at the shape and composition of our dataset.

In [24]:
# Displays the dimensions of the dataset
print(food.shape)

# Gives the number of unique restaurants
print(len(food['DBA'].unique()))

(211842, 18)
19616


So our tidied dataset consists of 211,842 rows and 18 columns with 19,616 distinct restaurants. 
If our dataset wasn't cleaned, it would've 451,674 rows and 20,260 distinct restaurants. So our current dataset dropped 239,832 rows (53% of the original data) and 644 restaurants.

Now, let's take a closer look at the data.

In [51]:
# The describe function gives us useful statistics on the Score of the dataset. 
food['SCORE'].describe()

count    211842.000000
mean         13.217478
std           8.220735
min           2.000000
25%           9.000000
50%          12.000000
75%          13.000000
max         131.000000
Name: SCORE, dtype: float64

According to New York City's Restaurant grading system, "restaurants with a score between 0 and 13 points earn an A, those with 14 to 27 points receive a B and those with 28 or more a C."

Luckily in New York City the majority of restaurants score enough to earn an A. Interestingly, the function reports a restaurant scoring above a 28. Even with such an outlier score of 131, the median and mean of the dataset are not too far apart.
Furthermore, the standard deviation of the score is rather low which explains that scores are mostly consistent.

Let's take a look at each of the 5 borough's scores.

In [52]:
# Groups the data by Borough and counts the number of unique restaurants for each boro
boros = food.groupby('BORO')
boros['DBA'].nunique().sort_values(ascending=False)

BORO
MANHATTAN        7923
BROOKLYN         5172
QUEENS           4736
BRONX            1831
STATEN ISLAND     771
Name: DBA, dtype: int64

Note that the above does not total up to 19,616 but rather 20,433. A good guess is that restaurants inspected with the same name are located in different locations. For example, the dataset can have one Dunkin Donuts in Staten Island and another in Queens. This can explain why the number of unique values aren't equivalent.

The borough with the least recorded information is Staten Island and the most recorded is Manhattan. 

Now we know that the majority of the dataset consists of info from Manhattan, Brooklyn, and Queens, which is frankly unsurprising. We'll now observe the count for each letter score for each borough.

In [53]:
# Create a new grouped object that groups by the borough and grade
boros_grade = food[['DBA','BORO', 'GRADE', 'SCORE']].groupby(['BORO', 'GRADE'])

# Use .size() to specifically count each individual object within the GRADE column
boros_grade['GRADE'].size()

BORO           GRADE         
BRONX          A                 14744
               B                  3490
               C                   792
               Not Yet Graded      114
               Pending             610
BROOKLYN       A                 38039
               B                  8585
               C                  2100
               Not Yet Graded      472
               Pending            1369
MANHATTAN      A                 64261
               B                 13699
               C                  3933
               Not Yet Graded      485
               Pending            2108
QUEENS         A                 37293
               B                  8955
               C                  2238
               Not Yet Graded      214
               Pending            1272
STATEN ISLAND  A                  5448
               B                  1060
               C                   287
               Not Yet Graded       69
               Pending            

Unfortunately, the size function is counting multiple copies of each distinct restaurant. Also, some restaurants' scores may change over time. For instance, one restaurant may have a C and improve itself into an A later on.
We can query the unique restaurants with its letter grade, but the dataset might have each distinct restaurant having multiple reports of the same grade.

Below, we'll use our grouped series to count the total of distinct restaurants with its corresponding letter grade.

In [67]:
# We'll keep the size function for comparison's sake.
boros_agg = boros_grade.agg({ 'GRADE' : "size", 
                             'DBA' : pd.Series.nunique})
boros_agg

DBA  GRADE
BORO          GRADE                      
BRONX         A               1734  14744
              B                608   3490
              C                170    792
              Not Yet Graded    51    114
              Pending          192    610
BROOKLYN      A               4900  38039
              B               1513   8585
              C                431   2100
              Not Yet Graded   183    472
              Pending          427   1369
MANHATTAN     A               7636  64261
              B               2442  13699
              C                708   3933
              Not Yet Graded   190    485
              Pending          629   2108
QUEENS        A               4534  37293
              B               1538   8955
              C                411   2238
              Not Yet Graded    83    214
              Pending          394   1272
STATEN ISLAND A                721   5448
              B                217   1060
              C                 55    287
              Not Yet Graded    28     69
              Pending           54    205

It looks much more understandable, but we still run into the problem of duplication. There are two problems occuring on our DBA column. 

The first is, as mentioned, how restaurants with the same name can be in multiple boroughs. The second is that one restaurant's letter grade can change over time. Say one restaurant scored a grade of C, but after a reinspection it earned itself an A. Our code does not account for these duplicates, and will lead to inflated values.

An easy solution would've been to use the delete_duplicates function when we were cleaning the data. For learning sake, we will just keep it in the back of our minds that the data has some inflated values. 

We'll use percentages to see how each borough's grade relates to one another.

In [74]:
#total = [2755, 7454, 11605, 6960, 1074]
grade_count = [[1734,608,170,51,192], [4900,1513,431,183,427], [7637,2442,708,190,629], 
               [4534,1538,411,83,394], [721,217,55,28,54]]

new_list = [[num/sum(i) for num in i] for i in grade_count]

grade_perc = {'Bronx' : new_list[0],
              'Brooklyn' : new_list[1],
             'Manhattan' : new_list[2],
             'Queens' : new_list[3],
             'Staten Island' : new_list[4]}
indexgrade = ['A', 'B', 'C', 'Not Yet Graded', 'Pending']

grade_perc = pd.DataFrame(grade_perc, index=indexgrade)
grade_perc

,Bronx,Brooklyn,Manhattan,Queens,Staten Island
A,0.629401,0.657365,0.658022,0.651437,0.670698
B,0.220690,0.202978,0.210408,0.220977,0.201860
C,0.061706,0.057821,0.061003,0.059052,0.051163
Not Yet Graded,0.018512,0.024551,0.016371,0.011925,0.026047
Pending,0.069691,0.057285,0.054196,0.056609,0.050233


Keep in mind that these percentages are influenced by the size of the duplicates and its borough total.
Still, the fact that each borough have about 60% of its restaurants achieving A's is a positive note. Viewing Brooklyn, Manhattan, and Queens, it is somewhat surprising that their A to C grading is very similar considering the difference in sample size.     

Now, let's check how spreadout each borough's numeric scores is. We will use the var function.

In [39]:
# Calculates the score variance
boros_grade.var()

SCORE
BORO          GRADE                     
BRONX         A                 7.875244
              B                14.800783
              C               197.222618
              Not Yet Graded  216.918180
              Pending         252.296398
BROOKLYN      A                 7.929569
              B                16.065505
              C               181.170943
              Not Yet Graded  373.941879
              Pending         165.993386
MANHATTAN     A                 7.771632
              B                16.165909
              C               194.341463
              Not Yet Graded  327.695467
              Pending         153.707721
QUEENS        A                 7.384179
              B                14.639380
              C               156.487357
              Not Yet Graded  558.995415
              Pending         151.351805
STATEN ISLAND A                 7.100574
              B                13.548877
              C               119.159597
              Not Yet Graded  221.890452
              Pending          86.644094

The variance of the number scores make sense. Grades with "Not Yet Graded" and "Pending" are scored numerically, but aren't categorized yet in A, B, or C. Take a look at the below example.

In [54]:
# Uses the keys to locate the "Not Yet Graded" category
groups = dict(list(boros_grade))
groups['QUEENS', 'Not Yet Graded'].head(5)

,DBA,BORO,GRADE,SCORE
3470,HOUSE OF BLESSED FOODS,QUEENS,Not Yet Graded,72
4942,LA KUCHARA,QUEENS,Not Yet Graded,10
8347,FISH DUMPLING,QUEENS,Not Yet Graded,2
10580,EL REY IV BILLIARD & LOUNGE,QUEENS,Not Yet Graded,12
12853,MCFADDENS,QUEENS,Not Yet Graded,41


The Fish Dumpling restaurant has a score of 2 while House of Blessed Foods has a 72 (yikes!). This type of disparity in these two grades in each borough will result in high variance. Even grade C is shown to have a relatively large variance because restaurants can score above a 28 (e.g. House of Blessed Foods). Restaurants with grades A and B will score a low variance because there is a set integer cutoff point.